In [1]:
using PorousMaterials

In [2]:
# # location of origional crystal structures used for lammps sims
# @eval PorousMaterials PATH_TO_CRYSTALS = joinpath("../mof_construction/NiPyC2_relax_sc211/")

**NOTE:** Helpful information about how "BOX" parameters are stored in the LAMMPS `*_restart.lammpstr` file can be found [here](https://lammps.sandia.gov/threads/msg10124.html)

In [3]:
function lammps_output_to_crystal(structurename::String)
    filename = structurename * "_restart.lammpstrj"

    lines = readlines(filename)

    ###
    #   get the Box
    ###
    # ITEM: BOX BOUNDS xy xz yz
    # xlo_bound xhi_bound xy
    # ylo_bound yhi_bound xz
    # zlo_bound zhi_bound yz
    xlo = parse(Float64, split(lines[6])[1])
    xhi = parse(Float64, split(lines[6])[2])
    xy = parse(Float64, split(lines[6])[3])

    ylo = parse(Float64, split(lines[7])[1])
    yhi = parse(Float64, split(lines[7])[2])
    xz = parse(Float64, split(lines[7])[3])

    zlo = parse(Float64, split(lines[8])[1])
    zhi = parse(Float64, split(lines[8])[2])
    yz = parse(Float64, split(lines[8])[3])

    #>>> a = (xhi-xlo,0,0)
    #>>> b = (xy,yhi-ylo,0)
    #>>> c = (xz,yz,zhi-zlo)
    a = [xhi - xlo, 0, 0]
    b = [xy, yhi-ylo, 0]
    c = [xz, yz, zhi-zlo]

    f_to_c = [a b c]
    box = Box(f_to_c)

    ###
    #  get number of atoms
    ###
    n_atoms = parse(Int, lines[4])

    ###
    #  get atom coords from xyz file
    #   (not from LAMMPS output file b/c types are opaque)
    ###
    xyz_filename = structurename * "_mov.xyz"
    run(pipeline(`tail -$(n_atoms+2) $xyz_filename`, "temp.xyz"))
    atoms = read_xyz("temp.xyz")
    rm("temp.xyz")
    # x = zeros(3, n_atoms)
    # for i = 1:n_atoms
    #     x_i = split.(lines[9+i])[3:end]
    #     for k = 1:3
    #         x[k, i] = parse(Float64, x_i[k])
    #     end
    # end

    ###
    #  construct Crystal
    ###
    # Crystal(name::String, box::Box, atoms::Atoms{Frac}, charges::Charges{Frac})
    return Crystal(structurename * "_lammps", box, Frac(atoms, box), Charges{Frac}(0))
end

# test if function works
# structurename = "NiPyC2_relax_sc211_meta_functionalized_C-3CH3"

# xtal = lammps_output_to_crystal(structurename)

# # xtal = replicate(xtal, (2,2,2))
# write_cif(xtal, structurename * "_lammps.cif")

lammps_output_to_crystal (generic function with 1 method)

In [4]:
# # The name of the fragment
# # file must be a .xyz file located in ./fragments/
# # The atom species that is bonded to the
# # :C_aro_R atom on the ring segment of the Fragment
fragments_with_overlap = ["C-3CH3", "CH2-CH2-CH3", "CH2-CH3","CH2-NH2",
                          "CH-CH2", "CH-S", "N-2CH3", "O-CH2-CH2-CH3",
                          "O-CH2-CH3", "O-CH3", "P-2CH3", "PH2", 
                          "S-CH3", "SH"]

# # The type of Arene Substitution ("ortho", "meta", "all") 
# # TODO: "all" not yet implimented
substitution_types = ["meta", "ortho"]

2-element Array{String,1}:
 "meta"
 "ortho"

In [6]:
for fragment in fragments_with_overlap
    # move into dir containing lammps sims of fragment
    cd(fragment)
    println(pwd())
    for sub_type in substitution_types
        structurename = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" * fragment
        xtal = lammps_output_to_crystal(structurename)
        write_cif(xtal, structurename * "_lammps.cif")
        overlap_flag, overlap_pairs = overlap(xtal)
        @assert !overlap_flag
        strip_numbers_from_atom_labels!(xtal)
        write_xyz(xtal)
        println(xtal.name)
    end
    # move back into parent dir
    cd("..")
    @assert pwd() == "/home/ng/DTRA/lammps_sims"
end    

/home/ng/DTRA/lammps_sims/C-3CH3
NiPyC2_relax_sc211_meta_functionalized_C-3CH3_lammps
NiPyC2_relax_sc211_ortho_functionalized_C-3CH3_lammps
/home/ng/DTRA/lammps_sims/CH2-CH2-CH3
NiPyC2_relax_sc211_meta_functionalized_CH2-CH2-CH3_lammps
NiPyC2_relax_sc211_ortho_functionalized_CH2-CH2-CH3_lammps
/home/ng/DTRA/lammps_sims/CH2-CH3
NiPyC2_relax_sc211_meta_functionalized_CH2-CH3_lammps
NiPyC2_relax_sc211_ortho_functionalized_CH2-CH3_lammps
/home/ng/DTRA/lammps_sims/CH2-NH2
NiPyC2_relax_sc211_meta_functionalized_CH2-NH2_lammps
NiPyC2_relax_sc211_ortho_functionalized_CH2-NH2_lammps
/home/ng/DTRA/lammps_sims/CH-CH2
NiPyC2_relax_sc211_meta_functionalized_CH-CH2_lammps
NiPyC2_relax_sc211_ortho_functionalized_CH-CH2_lammps
/home/ng/DTRA/lammps_sims/CH-S
NiPyC2_relax_sc211_meta_functionalized_CH-S_lammps
NiPyC2_relax_sc211_ortho_functionalized_CH-S_lammps
/home/ng/DTRA/lammps_sims/N-2CH3
NiPyC2_relax_sc211_meta_functionalized_N-2CH3_lammps
NiPyC2_relax_sc211_ortho_functionalized_N-2CH3_lammps
/home

In [7]:
# xtal.box
# xtal.box.f_to_c